## An Agent-based model of economics of space colonies

In this notebook we will create a rather simple look at economics between space colonies using an Agent-based model (ABM). ABMs are systems where individual organisms called agents follow a set of rules over a duration of a simulation. Mathematical simulations are often based on a set of equations describing an equilibrium of a system and studying effects leading to that point. ABMs by contrast work individually, often giving form to more complex phenomena if the agents are correctly set. This enables us to observe _emergence_ or the _"whole is greater than the sum of its parts"_ effect for complex systems.

This simulation creates a simplified universe with *planets* and *colonies*, each of which is situated in a 2d space. We want to study how constraints on diminishing resources create a network of trade in this universe. The universe consists of only two materials

- Commonium, a common material that is videly available on planets and is relatively cheap
- Importatium, a relatively rare mineral that is used for high-tech development

Additionally, each planet and colony will have the following traits

- Habitability, a number between 0 and 100. The number represents how hostile the area is for permanent settlementation, either by humans or automata. Each unit of habitability under 100 increases the cost of running the settlement. Money can be spent to increase habitability of the area. Habitability decreases with consumption of resources.
- Cost of operation: Cost to run the station, derived from habitability. Commonium and importantium are consumed for each person in the system.
- Population. Population consumes resources and creates wealth for the planet. 
- Capital, the number of machinery on the planet. Capital is increased over time with a known constant.
- Production. We will assume each planet porudces nothing but spaceships. A long shot, but this is a simple model. 

We use Cobb-Douglas productivity model (https://assets.aeaweb.org/asset-server/journals/aer/top20/18.1.139-165.pdf) of $Y = AK^\alpha L^\beta$, where $Y$ is the production output, $A$ is the scaling factor (often denoted productivity or technological prowess, which grows steadily over time), $K$ the amount of capital, $L$ the labour (equal to the population in our simulation) and $\alpha, \beta$ are constants for how much the output grows when one unit of capital or labor are added. Cobb and Douglas used values of $\alpha = 1/4, \beta = 3/4$ so we will run with those. A certain level of production is needed for spaceships, and spaceships wear down with time. They allow transportation of goods and people between planets. Spaceships have the following traits:

- Cargo: We assume 10 tonnes of payload, with each person and his belongings weighting a tonne.
- Cost to operate: a fixed sum of capital needed to run the spaceships
- Speed: how long it takes to reach each planet or settlement

The model is simple, so we will ignore most real-world restrictions. For example, we ignore the movement of planets, live on a 2d plane and disregard politics and sociology. We assume that the only motivation for humankind is to increase production indefinitely. The model aims to give a glimpse on how restricted resources give rise to colonies and the traffic flow between them

This is not a factual model, rather a proof-of-concept designed to play around with dynamics of space colonies. Much further work is needed to derive meaningful real-world implications from this work.

In [165]:
import random
import string
import pandas as pd

import math

import plotly.express as px

# Set repeatability
random.seed(358)

# Global variables
A = 1 # A in Cobb-Douglas formula. For this project, A is also a proxy of human advanceness. It increases productivity but also the price and consumption of goods
alpha = .25
beta = .75
cost_of_habitability = 100 # Production cost to offset one unit of habitability
cost_of_improving_habitability = 1000 # Number of production needed to improve habitability by 1
cost_of_breeding = A * 5
population_consumption = .2 # How much production a single person uses
deprecation_per_age = .1 # How much of capital is deprecated (ie. machines break down, food is wasted)
population_growth_max = 1.3 # Coefficient of free exponential growth

# Store the spaceships in space somewhere
spaceships_in_space = []

class Planet:
    
    # Initialise the planet
    def __init__(self, name, x, y, habitability, population, capital, commonium, importantium):
        self.age = 0
        self.name = name
        self.x = x # Coordinates of the planet
        self.y = y
        self.habitability = habitability # 0...100, with 100 being a habitable planet, 0 being the complete opposite
        self.population = population 
        self.capital = capital # Capital is increased via production of goods
        self.commonium = commonium # number of tonnes of material on the planet
        self.importantium = importantium # number of tonnes of material on the planet
        self.production = A*self.capital**alpha*self.population**beta # production
        self.spaceships = []
        
    # Planets make autonomous decisions. Each planet has a few possibilities
    # It must consume goods 
    def run_step(self):
        
        self.age += 1
        
        # First we produce material
        self.production = A*(self.capital**alpha)*(self.population**beta)
        if self.population == 0 or self.capital == 0:
            self.production = 0
        # Production uses up the planet's materials
        self.commonium -= self.production
        
        if self.commonium < 0:
            self.production = self.commonium
            self.commonium = 0
            
        # Consume importantium for the amount of production
        self.importantium -= min(A/10 * self.production, self.production) # Importantium use increases with age, but never exceeds commonium usage
        
        if self.importantium < 0:
            self.production = self.importantium*10/A
            self.importantium = 0
            
        
        # Next we feed the people. Carrying capacity is related to production
        carrying_capacity = self.production / population_consumption
        
        # If there are too many people, some unfortunately die off
        if carrying_capacity < self.population:
            self.population = math.floor(carrying_capacity)
        
        # Let us model a logistic growth model for population
        if carrying_capacity == 0:
            self.population = 0
        else:
            self.population += population_growth_max * (carrying_capacity - self.population) / carrying_capacity * self.population
        
        # Let population always be larger than 0
        self.population = max(self.population, 0)
        
        # Then we feed the population
        self.production -= self.population * population_consumption
        # We also use production to account for lack of habitability
        self.production -= self.population * (100 - self.habitability) * cost_of_habitability
        
        # The population is now sustained: all the production is used for increasing capital
        self.capital += self.production
        
        # Some capital is deprecated
        self.capital = self.capital * (1 - deprecation_per_age)
        
        self.production = max(0, self.production)
        self.capital = max(0, self.capital)
        
        # If we can make a spaceship you can be damn sure we make one
        cost_of_spaceship = max(10000, A/10*10000) # The price of spaceships increases as A increases
        
        while self.production >= cost_of_spaceship:
            self.spaceships.append(Spaceship())
            self.production -= cost_of_spaceship

    # ***** Getters *****
    def get_name(self):
        return self.name
    
    # Get the current status of the planet after a step
    def get_status(self, pandas = False):
        ret = {
            'age': self.age,
            'habitability': self.habitability,
            'population': self.population,
            'capital': self.capital,
            'commonium': self.commonium,
            'importantium': self.importantium,
            'production': self.production,
            'spaceships': len(self.spaceships)
        }
        
        if pandas:
            return pd.DataFrame(ret)
        else:
            return ret
        

    def to_dict(self):
        return {'name' : self.name, 'x' : self.x, 'y' : self.y, 'habitability' : self.habitability, 'population' : self.population, 'capital' : self.capital, 'commonium' : self.commonium, 'importantium' : self.importantium}

    
    def print_planet(self):
        print('Planet x: {}, y: {}, habitability: {}, population: {}, capital: {}, commonium: {}, importantium: {}\n'.format(self.x, self.y, self.habitability, self.population, self.capital, self.commonium, self.importantium))
    

class Spaceship:
    
    def __init__(self):
        self.speed = 100
        self.cargo = 10
        self.fuel = None
        self.cost_of_operation = 100
        self.location = None # None or Planet
        self.destination = None # When the ships are en route, this gets used to store the destination
        self.distance_to_destination = None
    
    def refuel(self, fuel):
        self.fuel += fuel
    
    def run_step(self):
        self.distance_to_destination -= self.speed
        # TODO: if we reach the destination, offload cargo, load new and off you go
        
        
class Universum:
    
    def __init__(self, n_planets):
        
        max_x = 100
        min_x = -100
        max_y = 100
        min_y = -100
        
        self.planets = []
        # Generate Terra
        self.planets.append(Planet(name = 'Terra', x = random.randint(min_x, max_x), y = random.randint(min_y, max_y), habitability = 100, population = 10, capital = 10, commonium = random.randint(100000,1000000), importantium = random.randint(100,10000)))
    
        for i in range(n_planets - 1):
            self.planets.append(Planet(name = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6)),x = random.randint(min_x, max_x), y = random.randint(min_y, max_y), habitability = random.randint(0,100), population = 0, capital = 0, commonium = random.randint(0,1000000), importantium = random.randint(0,1000000)))

    def get_planet_by_name(self, name):
        ids = None
        
        for idx, planet in enumerate(self.planets):
            if planet.get_name() == name:
                ids = idx
                
        if ids != None:
            return self.planets[ids]
        else:
            return None
        
            
    def get_planets(self):
        return self.planets
    
    def print_planets(self):
        for planet in self.planets:
            planet.print_planet()
            
    def to_pandas(self):
        i = 0
        list_of_planets = []
        for planet in self.planets:
            list_of_planets.append(planet.to_dict())
        return pd.DataFrame(list_of_planets)
    
    def visualize(self):
        fig = px.scatter(self.to_pandas(), x = 'x', y = 'y', size = 'habitability', hover_data = ['name'], color = 'population')
        fig.show()

Now we can create and visualize our universum.

In [166]:
universum = Universum(100)

universum.visualize()

Rather beautiful, if somewhat simple. Let's take a closer look at Terra and visualize the history of the 20 first generations.

In [167]:
terra = universum.get_planet_by_name("Terra")

n_to_run = 20
stats_all = []

for i in range(n_to_run):
    stats_all.append(terra.get_status())
    terra.run_step()
    
stats = pd.DataFrame(stats_all)

fig = px.line(stats.melt(id_vars = 'age'), x = 'age', y = 'value', color = 'variable')
fig.show()

Oh boy. It seems that humanity is set to doom itself with overconsumption. The consumption and population skyrocket until all the importantium is used up. Then it's ack to the stone age: humanity dies out over the course of one generation. Happily we seem to have a spaceship saved up!

In [168]:
stats

,age,capital,commonium,habitability,importantium,population,production,spaceships
0,0,10.000000,317145.000000,100,6510.000000,10.000000,10.000000,0
1,1,14.328000,317135.000000,100,6509.000000,20.400000,5.920000,0
2,2,22.300312,317116.324638,100,6507.132464,41.126184,10.450125,0
3,3,37.049062,317081.033428,100,6503.603343,82.129490,18.865312,0
4,4,64.610049,317013.725124,100,6496.872512,162.842115,34.739881,0
5,5,116.651632,316884.484196,100,6483.948420,321.190263,65.002875,0
6,6,215.784160,316635.142745,100,6459.014275,631.164524,123.108546,0
7,7,405.897652,316152.515510,100,6410.751551,1237.070022,235.213231,0
8,8,772.296719,315216.248229,100,6317.124823,2420.287339,452.209813,0
9,9,1480.926163,313397.191496,100,6135.219150,4729.399687,873.176795,0


Curiously our humanity only manages to build one. The humanity dies out the second we pop out the first spaceship. What if these people manage to send out a single rocket to save humanity?

In [ ]:
# TODO: tune the model to handle colonisation, and see how the universum is colonised.
# After that, develop trade between colonies. Clearly importantium is the first to go 
# out - what if we were to send some from the closest planets to Terra?

# Then we can set up a system with several functional planets and colonies. We can study the trade and
# material flow from there

# An interesting idea: could a similar model be usable also for colonisation of Americas in the lte 1400's
# and 1500's, or the American colonies from 1600 onwards?